Lectura de xml

In [1]:
import pandas as pd
import xml.etree.ElementTree as etree



tree = etree.parse("pu-cs-fal07.xml")
#tree = etree.parse("agh-ggis-spr17.xml")
root = tree.getroot()

columns_problem = ["Name", "NrDays", "SlotsPerDay", "NrWeeks"]
columns_optimization = ["Time", "Room", "Distribution", "Student"]
columns_rooms = ["RoomId", "Capacity", "PreviousRoomIdList", "TravelTimeValueList", "UnavailableDaysList", "UnavailableStartList", "UnavailableLengthList", "UnavailableWeeksList"]
columns_courses = ["CourseId", "ConfigIdList"]
columns_configs = ["ConfigId", "SubpartIdList"]
columns_subparts = ["SubpartId", "ClassIdList"]
columns_classes = ["ClassId", "ClassLimit", "ClassParent"]
columns_distributions = ["DistributionId", "DistributionType", "DistributionRequired", "DistributionPenalty", "ClassesList"]
columns_students = ["StudentId", "CoursesList"]
columns_relation_class_room = ["ClassId", "RoomId", "RoomPenalty"]
columns_relation_class_time = ["ClassId", "TimeDays", "TimeStart", "TimeLength", "TimeWeeks", "TimePenalty"]

df_problem = pd.DataFrame(columns = columns_problem)
df_optimization = pd.DataFrame(columns = columns_optimization)
df_rooms = pd.DataFrame(columns = columns_rooms)
df_courses = pd.DataFrame(columns = columns_courses)
df_configs = pd.DataFrame(columns = columns_configs)
df_subparts = pd.DataFrame(columns = columns_subparts)
df_classes = pd.DataFrame(columns = columns_classes)
df_distributions = pd.DataFrame(columns = columns_distributions)
df_students = pd.DataFrame(columns = columns_students)
df_relation_class_room = pd.DataFrame(columns = columns_relation_class_room)
df_relation_class_time = pd.DataFrame(columns = columns_relation_class_time)




df_problem = df_problem.append({"Name": root.attrib.get("name"), "NrDays": root.attrib.get("nrDays"), "SlotsPerDay": root.attrib.get("slotsPerDay"), "NrWeeks": root.attrib.get("nrWeeks")}, ignore_index = True)

for node in root:
    if node.tag == "optimization":
        df_optimization = df_optimization.append({"Time": node.attrib.get("time"), "Room": node.attrib.get("room"), "Distribution": node.attrib.get("distribution"), "Student": node.attrib.get("student")}, ignore_index = True)
    
    elif node.tag == "rooms":
        for room in node:
            roomid = room.attrib.get("id")
            cap = room.attrib.get("capacity")
            prevroomlist = []
            valuelist = []
            dayslist = []
            startlist = []
            lengthlist = []
            weekslist = []
            if len(list(child.tag for child in room.iter() if child is not room)) > 0:
                for child in room:
                    if child.tag == "travel":
                        prevroomid = child.attrib.get("room")
                        value = child.attrib.get("value")
                        prevroomlist.append(prevroomid)
                        valuelist.append(value)
                    elif child.tag == "unavailable":
                        days = child.attrib.get("days")
                        start = child.attrib.get("start")
                        length = child.attrib.get("length")
                        weeks = child.attrib.get("weeks")
                        dayslist.append(days)
                        startlist.append(start)
                        lengthlist.append(length)
                        weekslist.append(weeks)
            if len(prevroomlist) == 0:
                prevroomlist = None
            if len(valuelist) == 0:
                valuelist = None
            if len(dayslist) == 0:
                dayslist = None
            if len(startlist) == 0:
                startlist = None
            if len(lengthlist) == 0:
                lengthlist = None
            if len(weekslist) == 0:
                weekslist = None
            df_rooms = df_rooms.append(pd.Series([roomid, cap, prevroomlist, valuelist, dayslist, startlist, lengthlist, weekslist], index = columns_rooms), ignore_index = True)
            
    elif node.tag == "courses":
        for course in node:
            #print(course.tag)
            courseid = course.attrib.get("id")
            configlist = []
            for config in course:
                #print(config.tag)
                configid = config.attrib.get("id")
                configlist.append(configid)
                subpartlist = []
                for subpart in config:
                    #print(subpart.tag)
                    subpartid = subpart.attrib.get("id")
                    subpartlist.append(subpartid)
                    claselist = []
                    for clase in subpart:
                        #print(clase.tag)
                        claseid = clase.attrib.get("id")
                        claselist.append(claseid)
                        claselimit = clase.attrib.get("limit")
                        claseparent = clase.attrib.get("parent") if clase.attrib.get("parent") is not None else None
                        df_classes = df_classes.append(pd.Series([claseid, claselimit, claseparent], index = columns_classes), ignore_index = True)
                        if len(list(child.tag for child in clase.iter() if child is not clase)) > 0:
                            for child in clase:
                                #print(child.tag)
                                if child.tag == "room":
                                    roomid = child.attrib.get("id")
                                    roompenalty = child.attrib.get("penalty")
                                    df_relation_class_room = df_relation_class_room.append(pd.Series([claseid, roomid, roompenalty], index = columns_relation_class_room), ignore_index = True)
                                elif child.tag == "time":
                                    timedays = child.attrib.get("days")
                                    timestart = child.attrib.get("start")
                                    timelength = child.attrib.get("length")
                                    timeweeks = child.attrib.get("weeks")
                                    timepenalty = child.attrib.get("penalty")
                                    df_relation_class_time = df_relation_class_time.append(pd.Series([claseid, timedays, timestart, timelength, timeweeks, timepenalty], index = columns_relation_class_time), ignore_index = True)
                    df_subparts = df_subparts.append(pd.Series([subpartid, claselist], index = columns_subparts), ignore_index = True)
                df_configs = df_configs.append(pd.Series([configid, subpartlist], index = columns_configs), ignore_index = True)
            df_courses = df_courses.append(pd.Series([courseid, configlist], index = columns_courses), ignore_index = True)
                                    
    elif node.tag == "distributions":
        contador = 1
        for distribution in node:
            #print(distribution.tag)
            distribid = contador
            distribtype = distribution.attrib.get("type")
            distribrequired = distribution.attrib.get("required") if distribution.attrib.get("required") is not None else None
            distribpenalty = distribution.attrib.get("penalty") if distribution.attrib.get("penalty") is not None else None
            listaclases = []
            for clase in distribution:
                #print(clase.tag)
                listaclases.append(clase.attrib.get("id"))
            df_distributions = df_distributions.append(pd.Series([distribid, distribtype, distribrequired, distribpenalty, listaclases], index = columns_distributions), ignore_index = True)
            contador += 1
             
    elif node.tag == "students":
        for student in node:
            studentid = student.attrib.get("id")
            listacourses = []
            for course in student:
                listacourses.append(course.attrib.get("id"))
            df_students = df_students.append(pd.Series([studentid, listacourses], index = columns_students), ignore_index = True)
            
            


print("################################################################################################################")
print("DataFrame Problem")
display(df_problem)
print("################################################################################################################")
print("DataFrame Optimization")
display(df_optimization)
print("################################################################################################################")
print("DataFrame Rooms")
display(df_rooms)
print("################################################################################################################")
print("DataFrame Courses")
display(df_courses)
print("################################################################################################################")
print("DataFrame Configs")
display(df_configs)
print("################################################################################################################")
print("DataFrame Subparts")
display(df_subparts)
print("################################################################################################################")
print("DataFrame Classes")
display(df_classes)
print("################################################################################################################")
print("DataFrame Distributions")
display(df_distributions)
print("################################################################################################################")
print("DataFrame Students")
display(df_students)
print("################################################################################################################")
print("DataFrame RelationClassRoom")
display(df_relation_class_room)
print("################################################################################################################")
print("DataFrame RelationClassTime")
display(df_relation_class_time)
print("################################################################################################################")

################################################################################################################
DataFrame Problem


,Name,NrDays,SlotsPerDay,NrWeeks
0,pu-cs-fal07,7,288,15


################################################################################################################
DataFrame Optimization


,Time,Room,Distribution,Student
0,1,1,10,10


################################################################################################################
DataFrame Rooms


...

################################################################################################################
DataFrame RelationClassRoom


,ClassId,RoomId,RoomPenalty
0,1,10,0
1,1,11,0
2,1,12,0
3,2,10,0
4,2,11,0
...,...,...,...
385,173,2,0
386,173,3,0
387,174,1,44
388,174,2,0


################################################################################################################
DataFrame RelationClassTime


,ClassId,TimeDays,TimeStart,TimeLength,TimeWeeks,TimePenalty
0,1,0100000,90,22,111111111111111,0
1,1,0010000,90,22,111111111111111,0
2,1,0001000,90,22,111111111111111,0
3,1,0000100,90,22,111111111111111,0
4,1,0100000,114,22,111111111111111,0
...,...,...,...,...,...,...
2953,174,0101000,186,10,111111110000000,16
2954,174,1010000,198,10,111111110000000,16
2955,174,1000100,198,10,111111110000000,16
2956,174,0101000,198,10,111111110000000,16


################################################################################################################


Lectura xml solucion

Analisis de datos

In [3]:
## Duracion en slots de clases:

df_relation_class_time["TimeLength"] = df_relation_class_time["TimeLength"].astype(int)
print("Promedio:")
display(df_relation_class_time["TimeLength"].mean())
print("Min:")
display(df_relation_class_time["TimeLength"].min())
print("Max:")
display(df_relation_class_time["TimeLength"].max())
print("Tiempo de inicio")
display(df_relation_class_time.groupby("TimeStart").count())
x = df_relation_class_time.groupby("TimeStart").count()
print("Count")
display(df_relation_class_time["TimeLength"].value_counts())
print("Mediana:")
display(df_relation_class_time["TimeLength"].median())
print("Cuartiles:")
display(df_relation_class_time["TimeLength"].quantile([0, 0.25, 0.5, 0.75, 1]))
print("Var:")
display(df_relation_class_time["TimeLength"].var())
print("Std:")
display(df_relation_class_time["TimeLength"].std())

Promedio:


18.17816091954023

Min:


10

Max:


...

0.00    10.0
0.25    10.0
0.50    22.0
0.75    22.0
1.00    22.0
Name: TimeLength, dtype: float64

Var:


29.62093648812983

Std:


5.442511964904609

In [4]:
## Disponibilidad de salas en los días

for index, row in df_rooms.iterrows():
    #print(row["RoomId"], row["UnavailableDaysList"])
    if row["UnavailableDaysList"] is None:
        print(row["RoomId"], row["UnavailableDaysList"])

2 None
3 None
4 None
5 None
6 None
7 None
8 None
9 None
10 None
12 None
13 None


In [5]:
## Variabilidad de clases:

#for index, row in df_relation_class_time.iterrows():
#    print(row["ClassId"], row["TimeDays"])
display(df_relation_class_time["ClassId"].value_counts())

154    45
145    43
155    43
80     43
27     38
       ..
94      8
95      8
131     7
130     7
132     7
Name: ClassId, Length: 174, dtype: int64

In [6]:
##Slots de tiempo y cursos que duran esos slots
print("Slots de tiempo | Cantidad de clases que duran esos slots")
dic={}
#list=[]
for index, row in df_relation_class_time.iterrows():
    #print(row["ClassId"], row["TimeLength"])
    if row["TimeLength"] not in dic.keys():
        dic[row["TimeLength"]] = [0,[]]
    if row["ClassId"] not in dic[row["TimeLength"]][1]:
        #print(row["TimeLength"],row["ClassId"], dic[row["TimeLength"]][1])
        dic[row["TimeLength"]][0]+=1
        dic[row["TimeLength"]][1].append(row["ClassId"])
c= 0
for i in dic:
    print(i, dic[i][0])
    c += dic[i][0]
print("Total",c)

Slots de tiempo | Cantidad de clases que duran esos slots
22 106
10 68
15 24
Total 198


In [7]:
#########IGNORAR, código quizás útil para el futuro###############
#Patrones de salas según clases
diccr = {}
dic2 = {}
for index, row in df_relation_class_room.iterrows():
    #print(row["ClassId"], row["RoomId"])
    if row["ClassId"] not in diccr.keys():
        diccr[row["ClassId"]] = []
    diccr[row["ClassId"]].append(row["RoomId"])
#print(diccr)

for i in diccr:
    #print(i, diccr[i])
    tupla = tuple(diccr[i])
    if tupla not in dic2.keys():
        dic2[tupla] = []
    dic2[tupla].append(i)
#print(dic2)

c = 0

#print("Salas | Preferencias de clases por la sala")
#for i in dic2:
#    print(i, dic2[i])
#    c+=1
#print(c)

In [8]:
#Agrupar capacidad de clases

print("Límite de las clases | Cantidad de clases con el límite")
display(df_classes.groupby("ClassLimit").count()["ClassId"])


Límite de las clases | Cantidad de clases con el límite


ClassLimit
15      2
18      8
20     15
21      2
22    104
24      1
27     15
30      6
36     11
44      1
46      1
54      1
60      4
8       3
Name: ClassId, dtype: int64

In [9]:
#Cantidad de clases en una subparte
c = 0
dics = {}
dicc_s = {}
for index, row in df_subparts.iterrows():
    dicc_s[row["SubpartId"]] = row["ClassIdList"]
    
for index, col in df_configs.iterrows():
    for i in col["SubpartIdList"]:
        if len(dicc_s[i]) not in dics.keys():
            dics[len(dicc_s[i])] = [1, [], [int(col["ConfigId"])]]
            for z in dicc_s[i]:
                dics[len(dicc_s[i])][1].append(z)
                dics[len(dicc_s[i])][0]+=1
        else:
            dics[len(dicc_s[i])][2].append(int(col["ConfigId"]))
            for z in dicc_s[i]:
                dics[len(dicc_s[i])][1].append(z)
                dics[len(dicc_s[i])][0]+=1


#print("Cantidad de clases en una subparte | Cantidad de veces que se repite en las subpartes | Lista de clases | Config a la que pertenece ")
#for i in sorted(dics):
#    print(i, dics[i][0])

In [10]:
#Cantidad de subpartes de una configuración

c = 0
dics = {}
dicc_s = {}
for index, row in df_subparts.iterrows():
    dicc_s[row["SubpartId"]] = row["ClassIdList"]
    
for index, col in df_configs.iterrows():
        #print(col["ConfigId"],col["SubpartIdList"])
        if len(col["SubpartIdList"]) not in dics.keys():
            dics[len(col["SubpartIdList"])] = [0, [], [int(col["ConfigId"])], 1]
            for z in col["SubpartIdList"]:
                dics[len(col["SubpartIdList"])][0]+=1
                for w in dicc_s[z]: 
                    dics[len(col["SubpartIdList"])][1].append(w)
                    dics[len(col["SubpartIdList"])][3]+=1
        else:
            dics[len(col["SubpartIdList"])][2].append(int(col["ConfigId"]))
            for z in col["SubpartIdList"]:
                dics[len(col["SubpartIdList"])][0]+=1
                for w in dicc_s[z]: 
                    dics[len(col["SubpartIdList"])][1].append(w)
                    dics[len(col["SubpartIdList"])][3]+=1


print("Cantidad de subpartes en una config | Cantidad de veces que se repite | Lista de clases | Config a la que pertenece ")
c=0
for i in sorted(dics):
        print(i, dics[i][0])
        c+=dics[i][0]
#print(dics)



Cantidad de subpartes en una config | Cantidad de veces que se repite | Lista de clases | Config a la que pertenece 
1 24
2 40


In [11]:
import math
#Agrupar capacidad de cursos

#print("Límite de las clases | Cantidad de clases con el límite")
#display(df_classes.groupby("ClassLimit").count()["ClassId"])

dicc_cursos = {}
#display(df_courses)
for index, row in df_courses.iterrows():
    dicc_cursos[row["CourseId"]] = row["ConfigIdList"]
#print(dicc_cursos)      
dicc_configs = {}
#display(df_configs)
for index, row in df_configs.iterrows():
    dicc_configs[row["ConfigId"]] = row["SubpartIdList"]
#print(dicc_configs)
dicc_subparts = {}
#display(df_subparts)
for index, row in df_subparts.iterrows():
    dicc_subparts[row["SubpartId"]] = row["ClassIdList"]
#print(dicc_subparts)
dicc_classes = {}
#display(df_classes)
for index, row in df_classes.iterrows():
    dicc_classes[row["ClassId"]] = row["ClassLimit"]
#print(dicc_classes)

dicc_lim = {}
dicc_curso_clase = {}
for p,i in dicc_cursos.items():
    for j in i:
        if j in dicc_configs.keys():
            #print(dicc_configs[j])
            for k in dicc_configs[j]:
                if k in dicc_subparts.keys():
                    #print(dicc_subparts[k])
                    for l in dicc_subparts[k]:
                        #print(l)
                        #Agregué estas lineas para hacerme un diccionario de curso con clase#
                        if l not in dicc_curso_clase:
                            dicc_curso_clase[l] = []
                        dicc_curso_clase[l].append(p)
                        ######################################################################
                        if l in dicc_classes.keys():
                            n = int(dicc_classes[l])
                            if p not in dicc_lim.keys():
                                dicc_lim[p] = [n]
                            else:
                                dicc_lim[p].append(n)
                        
#print(dicc_lim)
#print("Id de curso | Limite de alumnos")
for i in dicc_lim.keys():
    suma = 0
    contador = 0
    for j in dicc_lim[i]:
        suma += j
        contador += 1
    promedio = suma/contador
    dicc_lim[i] = int(math.ceil(promedio/10.0))*10
   # print(i, dicc_lim[i])

In [12]:
# Slot de inicio y cantidad de clases en esos slots
d = {}
for index,row in df_relation_class_time.iterrows():
    #print(row["TimeStart"])
    if int(row["TimeStart"]) not in d.keys():
        d[int(row["TimeStart"])] = [0,[]]
    d[int(row["TimeStart"])][0]+=1
    d[int(row["TimeStart"])][1].append(row["ClassId"])
#print(sorted(d.items()))
print("Slot de inicio | Conteo de clases")
#for i in sorted(d.items()):
#    print(i[0],i[1][0])


Slot de inicio | Conteo de clases


In [13]:
#Slot de inicio y cantidad de cursos en esos slots
print("Horario de inicio | Cantidad de cursos")
di = {}

for i in sorted(d.items()):
    #print(i[0],i[1][1])
    for j in set(i[1][1]):
        if i[0] not in di.keys():
            di[i[0]] = [0,[]]
        curso = dicc_curso_clase[j]
        if curso not in di[i[0]][1]:
            di[i[0]][0]+=1
            di[i[0]][1].append(curso)
print(di)

Horario de inicio | Cantidad de cursos
{90: [7, [['1'], ['14'], ['4'], ['3'], ['5'], ['2'], ['43']]], 102: [36, [['26'], ['15'], ['40'], ['9'], ['27'], ['35'], ['4'], ['5'], ['18'], ['22'], ['7'], ['36'], ['12'], ['13'], ['31'], ['24'], ['23'], ['39'], ['19'], ['10'], ['29'], ['42'], ['28'], ['30'], ['38'], ['2'], ['11'], ['21'], ['34'], ['43'], ['37'], ['32'], ['6'], ['41'], ['8'], ['25']]], 108: [22, [['26'], ['27'], ['35'], ['22'], ['36'], ['31'], ['23'], ['39'], ['16'], ['19'], ['29'], ['42'], ['30'], ['38'], ['34'], ['43'], ['37'], ['32'], ['6'], ['33'], ['25'], ['41']]], 114: [44, [['6'], ['26'], ['15'], ['27'], ['1'], ['14'], ['4'], ['5'], ['36'], ['3'], ['10'], ['21'], ['20'], ['28'], ['38'], ['34'], ['11'], ['37'], ['33'], ['12'], ['25'], ['13'], ['40'], ['16'], ['7'], ['31'], ['23'], ['42'], ['9'], ['2'], ['17'], ['18'], ['24'], ['39'], ['29'], ['19'], ['8'], ['35'], ['22'], ['44'], ['30'], ['43'], ['32'], ['41']]], 126: [39, [['26'], ['15'], ['40'], ['18'], ['9'], ['27'], ['

In [14]:
## cursos más demandados por estudiantes
dic_demanda_cursos = {}
for index,row in df_students.iterrows():
    for i in row["StudentId"]:
        for j in row["CoursesList"]:
            if j not in dic_demanda_cursos.keys():
                dic_demanda_cursos[j] = 1
                #print("agregado:",j,dic_demanda_cursos[j])
            else:
                dic_demanda_cursos[j] += 1
                #print("suma +1:",j,dic_demanda_cursos[j])
#print("Id Curso | Cantidad demandada")
#for w in dic_demanda_cursos.keys():    
#    print(w,dic_demanda_cursos[w])

In [15]:
## Conteo de las salas mas utilizadas

dicc_conteo_clases_sala = {}
for index, row in df_relation_class_room.iterrows():
    if row["RoomId"] not in dicc_conteo_clases_sala.keys():
        dicc_conteo_clases_sala[row["RoomId"]] = 1
    else:
        dicc_conteo_clases_sala[row["RoomId"]] += 1
#print(dicc_conteo_clases_sala)
#print(len(dicc_conteo_clases_sala.keys()))
print("Sala | Cantidad de veces que se solicita")
#{k: v for k, v in sorted(dicc_conteo_clases_sala.items(), key=lambda item: item[1])}
#for i in sorted(dicc_conteo_clasess_sala)

Sala | Cantidad de veces que se solicita


In [16]:
## Cuantos cursos hay para cada capacidad redondeada

print("Capacidad de curso | Conteo de cursos")
dicc_cont = {}
for i in dicc_lim.values():
        if i not in dicc_cont.keys():
            dicc_cont[i] = 1
        else:
            dicc_cont[i] += 1
for i in dicc_cont.keys():
    print(i, dicc_cont[i])

Capacidad de curso | Conteo de cursos
30 25
40 9
20 8
60 1
50 1


In [40]:
##################### GUROBI #######################

In [17]:
from gurobipy import *
from random   import *
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [248]:
timetabling = gp.Model("Asignacion de cursos a horarios y salas")

#indices por variable
# x: (cluster,config,bloque)
# w: (cluster,curso)
# c: (config,bloque,patron)


#diccionario cluster {id cluster: cursos dentro}
# config {id curso: }
# conjuntos/subconjuntos
########################
C = dicc_cluster
B = dicc_bloques
Z = dicc_salas
D = {1:"Lunes", 2: "Martes", 3: "Miercoles", 4: "Jueves", 5: "Viernes", 6: "Sabado", 7: "Domingo"}

#subconjuntos
PB_c = dicc_pb
D_c = dicc_dias2

# parámetros
dicc_dias = {1:"Lunes", 2: "Martes", 3: "Miercoles", 4: "Jueves", 5: "Viernes", 6: "Sabado", 7: "Domingo"}
Q_c = dicc_qc 
M = 100000000000
QN_zp = dicc_qnz
QS_z = dicc_qs
L_c = dicc_lc
h_c = 130
TP_c = dicc_tp  
h_cc = 60
#PB_c = dicc_pb
#print(Q_c)
## variables

x = {} 
for c in dicc_cluster.keys():
    for d in dicc_dias.keys():
        for b in dicc_bloques.keys():
            x[c,d,b] = timetabling.addVar(vtype=gp.GRB.BINARY,name='x_'+str(c)+','+str(d)+','+str(b))
y = {}
for c in dicc_cluster.keys():
    for z in dicc_salas.keys():
        y[c,z] = timetabling.addVar(vtype=gp.GRB.BINARY,name='y_'+str(c)+','+str(z))


#Función Objetivo
N= 1 # ???:c
timetabling.setObjective(gp.quicksum(y[c,z] for c in C for z in Z),GRB.MINIMIZE)

#Restricciones

#1 3 5
#1 5 6
#1. Cada cluster de cursos c debe ser asignada a tipo de sala z
timetabling.addConstrs(((gp.quicksum(y[c,z] for z in Z) >= 1) for c in C), name = "R1")
###2. Cada cluster es asignado a tantas veces como clases requiera
timetabling.addConstrs(((gp.quicksum(x[c,d,b] for d in D_c[c] for b in PB_c[c]) >= Q_c[c]) for c in C), name = "R2")
#3. Si hay asignación de cluster a un horario, debe ser asignado también a un tipo de sala.
timetabling.addConstrs((x[c,d,b] == gp.quicksum(y[c,z] for z in Z) for c in C for d in D_c[c] for b in PB_c[c]), name = "R3")
#4. El tipo de salas alcance a satisfacer la cantidad de alumnos del cluster
timetabling.addConstrs(((y[c,z]*QS_z[z] - h_cc <= x[c,d,b]*L_c[c]) for c in C for b in PB_c[c] for d in D_c[c] for z in Z), name = "R4") 
##5. Limitar el tipo de salas mientras sea eficiente
timetabling.addConstrs(((h_c + QS_z[z]*y[c,z] >= x[c,d,b]*L_c[c]) for b in PB_c[c] for d in D for c in C for z in Z), name = "R5")
#6. Un cluster será asignado a tantos bloques como necesite
# Error en esta restriccion ocurre porque le estamos diciendo que tiene que ser asignado por ejemplo en 2 bloques dado
# que como parametro de duracion igual a 2 bloques, pero como tiene solo un bloque posible de inicio, no puede asignar 
# esos 2 bloques en un solo inicio
#timetabling.addConstrs(((gp.quicksum(x[c,d,b] for b in PB_c[c]) == TP_c[c]) for d in D_c[c] for c in C), name = "R6")
#7 nadie te quiere
## VER LOS BLOQUES 
#timetabling.addConstr((gp.quicksum(x[c,d,b] for b in PB_c[c]) == (gp.quicksum(x[c,d,b] for b in PB_c[c]) for d in D_c[c] for c in C), name = "R7")

timetabling.optimize()
for v in timetabling.getVars():
    print('%s %g' % (v.varName, v.x))
    #print('%s %g' % (v.varName, v.y))



Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 12360 rows, 14145 columns and 31578 nonzeros
Model fingerprint: 0x23ec3d24
Variable types: 0 continuous, 14145 integer (14145 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 115.0000000
Presolve removed 12360 rows and 14145 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 115 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.150000000000e+02, best bound 1.150000000000e+02, gap 0.0000%
x_0,1,1 0
x_0,1,2 0
x_0,1,3 0
x_0,1,4 0
x_0,1,5 0
x_0,1,6 0
x_0,1,7 0
x_0,1,8 0
x_0,1,9 0
x_0,1,10 0
x_0,1,11 0
x_0,1,12 0
x_0,1,13 0
x_0,1,14 0
x_0,1,15 0
x_0,1,16 0
x_0,1,17 0
x_0,2,1 0
x_0,2,2 0
x_0,2,3 0
x_0,2,4 0
x_0,2,

x_14,2,14 0
x_14,2,15 0
x_14,2,16 0
x_14,2,17 0
x_14,3,1 0
x_14,3,2 0
x_14,3,3 0
x_14,3,4 0
x_14,3,5 0
x_14,3,6 0
x_14,3,7 0
x_14,3,8 0
x_14,3,9 0
x_14,3,10 0
x_14,3,11 0
x_14,3,12 0
x_14,3,13 0
x_14,3,14 0
x_14,3,15 0
x_14,3,16 0
x_14,3,17 0
x_14,4,1 0
x_14,4,2 0
x_14,4,3 0
x_14,4,4 0
x_14,4,5 0
x_14,4,6 0
x_14,4,7 0
x_14,4,8 0
x_14,4,9 0
x_14,4,10 0
x_14,4,11 0
x_14,4,12 0
x_14,4,13 0
x_14,4,14 0
x_14,4,15 0
x_14,4,16 0
x_14,4,17 0
x_14,5,1 0
x_14,5,2 0
x_14,5,3 0
x_14,5,4 0
x_14,5,5 0
x_14,5,6 0
x_14,5,7 0
x_14,5,8 0
x_14,5,9 0
x_14,5,10 0
x_14,5,11 0
x_14,5,12 0
x_14,5,13 0
x_14,5,14 0
x_14,5,15 0
x_14,5,16 0
x_14,5,17 0
x_14,6,1 0
x_14,6,2 0
x_14,6,3 0
x_14,6,4 0
x_14,6,5 0
x_14,6,6 0
x_14,6,7 0
x_14,6,8 0
x_14,6,9 0
x_14,6,10 0
x_14,6,11 0
x_14,6,12 0
x_14,6,13 0
x_14,6,14 0
x_14,6,15 0
x_14,6,16 0
x_14,6,17 0
x_14,7,1 0
x_14,7,2 0
x_14,7,3 0
x_14,7,4 0
x_14,7,5 0
x_14,7,6 0
x_14,7,7 1
x_14,7,8 0
x_14,7,9 0
x_14,7,10 0
x_14,7,11 0
x_14,7,12 0
x_14,7,13 0
x_14,7,14 0
x_14,7,15 0
x

x_36,6,8 0
x_36,6,9 0
x_36,6,10 0
x_36,6,11 0
x_36,6,12 0
x_36,6,13 0
x_36,6,14 0
x_36,6,15 0
x_36,6,16 0
x_36,6,17 0
x_36,7,1 0
x_36,7,2 0
x_36,7,3 0
x_36,7,4 0
x_36,7,5 0
x_36,7,6 0
x_36,7,7 0
x_36,7,8 0
x_36,7,9 0
x_36,7,10 0
x_36,7,11 0
x_36,7,12 0
x_36,7,13 0
x_36,7,14 0
x_36,7,15 0
x_36,7,16 0
x_36,7,17 0
x_37,1,1 0
x_37,1,2 0
x_37,1,3 0
x_37,1,4 0
x_37,1,5 0
x_37,1,6 0
x_37,1,7 0
x_37,1,8 0
x_37,1,9 0
x_37,1,10 0
x_37,1,11 0
x_37,1,12 0
x_37,1,13 0
x_37,1,14 0
x_37,1,15 0
x_37,1,16 0
x_37,1,17 0
x_37,2,1 1
x_37,2,2 1
x_37,2,3 1
x_37,2,4 1
x_37,2,5 1
x_37,2,6 1
x_37,2,7 1
x_37,2,8 1
x_37,2,9 1
x_37,2,10 1
x_37,2,11 1
x_37,2,12 1
x_37,2,13 0
x_37,2,14 0
x_37,2,15 0
x_37,2,16 1
x_37,2,17 1
x_37,3,1 0
x_37,3,2 0
x_37,3,3 0
x_37,3,4 0
x_37,3,5 0
x_37,3,6 0
x_37,3,7 0
x_37,3,8 0
x_37,3,9 0
x_37,3,10 0
x_37,3,11 0
x_37,3,12 0
x_37,3,13 0
x_37,3,14 0
x_37,3,15 0
x_37,3,16 0
x_37,3,17 0
x_37,4,1 1
x_37,4,2 1
x_37,4,3 1
x_37,4,4 1
x_37,4,5 1
x_37,4,6 1
x_37,4,7 1
x_37,4,8 1
x_37,4,9 1
x_3

x_57,1,9 0
x_57,1,10 0
x_57,1,11 0
x_57,1,12 0
x_57,1,13 0
x_57,1,14 0
x_57,1,15 0
x_57,1,16 0
x_57,1,17 0
x_57,2,1 0
x_57,2,2 0
x_57,2,3 0
x_57,2,4 0
x_57,2,5 0
x_57,2,6 0
x_57,2,7 0
x_57,2,8 1
x_57,2,9 0
x_57,2,10 0
x_57,2,11 0
x_57,2,12 0
x_57,2,13 0
x_57,2,14 0
x_57,2,15 0
x_57,2,16 0
x_57,2,17 0
x_57,3,1 0
x_57,3,2 0
x_57,3,3 0
x_57,3,4 0
x_57,3,5 0
x_57,3,6 0
x_57,3,7 0
x_57,3,8 0
x_57,3,9 0
x_57,3,10 0
x_57,3,11 0
x_57,3,12 0
x_57,3,13 0
x_57,3,14 0
x_57,3,15 0
x_57,3,16 0
x_57,3,17 0
x_57,4,1 0
x_57,4,2 0
x_57,4,3 0
x_57,4,4 0
x_57,4,5 0
x_57,4,6 0
x_57,4,7 0
x_57,4,8 1
x_57,4,9 0
x_57,4,10 0
x_57,4,11 0
x_57,4,12 0
x_57,4,13 0
x_57,4,14 0
x_57,4,15 0
x_57,4,16 0
x_57,4,17 0
x_57,5,1 0
x_57,5,2 0
x_57,5,3 0
x_57,5,4 0
x_57,5,5 0
x_57,5,6 0
x_57,5,7 0
x_57,5,8 0
x_57,5,9 0
x_57,5,10 0
x_57,5,11 0
x_57,5,12 0
x_57,5,13 0
x_57,5,14 0
x_57,5,15 0
x_57,5,16 0
x_57,5,17 0
x_57,6,1 0
x_57,6,2 0
x_57,6,3 0
x_57,6,4 0
x_57,6,5 0
x_57,6,6 0
x_57,6,7 0
x_57,6,8 0
x_57,6,9 0
x_57,6,10 0
x_

x_77,3,3 0
x_77,3,4 0
x_77,3,5 0
x_77,3,6 0
x_77,3,7 0
x_77,3,8 0
x_77,3,9 0
x_77,3,10 0
x_77,3,11 0
x_77,3,12 0
x_77,3,13 0
x_77,3,14 0
x_77,3,15 0
x_77,3,16 0
x_77,3,17 0
x_77,4,1 0
x_77,4,2 0
x_77,4,3 0
x_77,4,4 0
x_77,4,5 0
x_77,4,6 0
x_77,4,7 0
x_77,4,8 0
x_77,4,9 0
x_77,4,10 0
x_77,4,11 0
x_77,4,12 0
x_77,4,13 1
x_77,4,14 0
x_77,4,15 0
x_77,4,16 0
x_77,4,17 0
x_77,5,1 0
x_77,5,2 0
x_77,5,3 0
x_77,5,4 0
x_77,5,5 0
x_77,5,6 0
x_77,5,7 0
x_77,5,8 0
x_77,5,9 0
x_77,5,10 0
x_77,5,11 0
x_77,5,12 0
x_77,5,13 0
x_77,5,14 0
x_77,5,15 0
x_77,5,16 0
x_77,5,17 0
x_77,6,1 0
x_77,6,2 0
x_77,6,3 0
x_77,6,4 0
x_77,6,5 0
x_77,6,6 0
x_77,6,7 0
x_77,6,8 0
x_77,6,9 0
x_77,6,10 0
x_77,6,11 0
x_77,6,12 0
x_77,6,13 0
x_77,6,14 0
x_77,6,15 0
x_77,6,16 0
x_77,6,17 0
x_77,7,1 0
x_77,7,2 0
x_77,7,3 0
x_77,7,4 0
x_77,7,5 0
x_77,7,6 0
x_77,7,7 0
x_77,7,8 0
x_77,7,9 0
x_77,7,10 0
x_77,7,11 0
x_77,7,12 0
x_77,7,13 0
x_77,7,14 0
x_77,7,15 0
x_77,7,16 0
x_77,7,17 0
x_78,1,1 0
x_78,1,2 0
x_78,1,3 0
x_78,1,4 0
x_7

x_85,3,7 1
x_85,3,8 1
x_85,3,9 1
x_85,3,10 1
x_85,3,11 1
x_85,3,12 1
x_85,3,13 1
x_85,3,14 1
x_85,3,15 1
x_85,3,16 1
x_85,3,17 0
x_85,4,1 0
x_85,4,2 0
x_85,4,3 0
x_85,4,4 0
x_85,4,5 0
x_85,4,6 0
x_85,4,7 0
x_85,4,8 0
x_85,4,9 0
x_85,4,10 0
x_85,4,11 0
x_85,4,12 0
x_85,4,13 0
x_85,4,14 0
x_85,4,15 0
x_85,4,16 0
x_85,4,17 0
x_85,5,1 1
x_85,5,2 1
x_85,5,3 1
x_85,5,4 1
x_85,5,5 1
x_85,5,6 1
x_85,5,7 1
x_85,5,8 1
x_85,5,9 1
x_85,5,10 1
x_85,5,11 1
x_85,5,12 1
x_85,5,13 1
x_85,5,14 1
x_85,5,15 1
x_85,5,16 1
x_85,5,17 0
x_85,6,1 0
x_85,6,2 0
x_85,6,3 0
x_85,6,4 0
x_85,6,5 0
x_85,6,6 0
x_85,6,7 0
x_85,6,8 0
x_85,6,9 0
x_85,6,10 0
x_85,6,11 0
x_85,6,12 0
x_85,6,13 0
x_85,6,14 0
x_85,6,15 0
x_85,6,16 0
x_85,6,17 0
x_85,7,1 0
x_85,7,2 0
x_85,7,3 0
x_85,7,4 0
x_85,7,5 0
x_85,7,6 0
x_85,7,7 0
x_85,7,8 0
x_85,7,9 0
x_85,7,10 0
x_85,7,11 0
x_85,7,12 0
x_85,7,13 0
x_85,7,14 0
x_85,7,15 0
x_85,7,16 0
x_85,7,17 0
x_86,1,1 0
x_86,1,2 1
x_86,1,3 1
x_86,1,4 1
x_86,1,5 1
x_86,1,6 0
x_86,1,7 0
x_86,1,8 0
x_8

x_97,3,7 1
x_97,3,8 1
x_97,3,9 1
x_97,3,10 1
x_97,3,11 1
x_97,3,12 1
x_97,3,13 1
x_97,3,14 1
x_97,3,15 1
x_97,3,16 1
x_97,3,17 0
x_97,4,1 1
x_97,4,2 1
x_97,4,3 1
x_97,4,4 1
x_97,4,5 1
x_97,4,6 1
x_97,4,7 1
x_97,4,8 1
x_97,4,9 1
x_97,4,10 1
x_97,4,11 1
x_97,4,12 1
x_97,4,13 1
x_97,4,14 1
x_97,4,15 1
x_97,4,16 1
x_97,4,17 0
x_97,5,1 1
x_97,5,2 1
x_97,5,3 1
x_97,5,4 1
x_97,5,5 1
x_97,5,6 1
x_97,5,7 1
x_97,5,8 1
x_97,5,9 1
x_97,5,10 1
x_97,5,11 1
x_97,5,12 1
x_97,5,13 1
x_97,5,14 1
x_97,5,15 1
x_97,5,16 1
x_97,5,17 0
x_97,6,1 0
x_97,6,2 0
x_97,6,3 0
x_97,6,4 0
x_97,6,5 0
x_97,6,6 0
x_97,6,7 0
x_97,6,8 0
x_97,6,9 0
x_97,6,10 0
x_97,6,11 0
x_97,6,12 0
x_97,6,13 0
x_97,6,14 0
x_97,6,15 0
x_97,6,16 0
x_97,6,17 0
x_97,7,1 0
x_97,7,2 0
x_97,7,3 0
x_97,7,4 0
x_97,7,5 0
x_97,7,6 0
x_97,7,7 0
x_97,7,8 0
x_97,7,9 0
x_97,7,10 0
x_97,7,11 0
x_97,7,12 0
x_97,7,13 0
x_97,7,14 0
x_97,7,15 0
x_97,7,16 0
x_97,7,17 0
x_98,1,1 1
x_98,1,2 1
x_98,1,3 1
x_98,1,4 1
x_98,1,5 1
x_98,1,6 1
x_98,1,7 1
x_98,1,8 1
x_9

y_65,S 0
y_65,XS 1
y_65,M 0
y_66,L 0
y_66,S 0
y_66,XS 0
y_66,M 1
y_67,L 0
y_67,S 1
y_67,XS 0
y_67,M 0
y_68,L 0
y_68,S 1
y_68,XS 0
y_68,M 0
y_69,L 0
y_69,S 0
y_69,XS 1
y_69,M 0
y_70,L 0
y_70,S 0
y_70,XS 1
y_70,M 0
y_71,L 0
y_71,S 0
y_71,XS 1
y_71,M 0
y_72,L 0
y_72,S 1
y_72,XS 0
y_72,M 0
y_73,L 0
y_73,S 0
y_73,XS 0
y_73,M 1
y_74,L 0
y_74,S 1
y_74,XS 0
y_74,M 0
y_75,L 0
y_75,S 1
y_75,XS 0
y_75,M 0
y_76,L 0
y_76,S 0
y_76,XS 1
y_76,M 0
y_77,L 0
y_77,S 0
y_77,XS 0
y_77,M 1
y_78,L 0
y_78,S 0
y_78,XS 1
y_78,M 0
y_79,L 0
y_79,S 0
y_79,XS 1
y_79,M 0
y_80,L 0
y_80,S 0
y_80,XS 0
y_80,M 1
y_81,L 0
y_81,S 0
y_81,XS 1
y_81,M 0
y_82,L 0
y_82,S 0
y_82,XS 0
y_82,M 1
y_83,L 0
y_83,S 1
y_83,XS 0
y_83,M 0
y_84,L 0
y_84,S 1
y_84,XS 0
y_84,M 0
y_85,L 0
y_85,S 1
y_85,XS 0
y_85,M 0
y_86,L 0
y_86,S 1
y_86,XS 0
y_86,M 0
y_87,L 0
y_87,S 0
y_87,XS 1
y_87,M 0
y_88,L 0
y_88,S 1
y_88,XS 0
y_88,M 0
y_89,L 0
y_89,S 1
y_89,XS 0
y_89,M 0
y_90,L 0
y_90,S 1
y_90,XS 0
y_90,M 0
y_91,L 0
y_91,S 1
y_91,XS 0
y_91,M 0
y_92,L 0
y

In [219]:
timetabling.write("file_infeasible.lp")
timetabling.write("file_infeasible.mps")

In [18]:
timetabling = gp.Model("Asignacion de cursos a horarios y salas")

#indices por variable
# x: (cluster,config,bloque)
# w: (cluster,curso)
# c: (config,bloque,patron)


#diccionario cluster {id cluster: cursos dentro}
# config {id curso: }
# conjuntos/subconjuntos
########################
C = dicc_cluster
B = dicc_bloques
Z = dicc_salas
D = {1:"Lunes", 2: "Martes", 3: "Miercoles", 4: "Jueves", 5: "Viernes", 6: "Sabado", 7: "Domingo"}
E = dicc_estudiantes

#subconjuntos
PB_c = dicc_pb
D_c = dicc_dias2

# parámetros
Lc=capacidad de estudiantes de la clase c
Qs=capacidad de estudiantes de la sala s
DTcd=duracion en slots de tiempo de la clase c en el dia d
QKe=cantidad de cursos demandados por el estudiante e
SAstd=1 si la sala s esta disponible en el slot t del dia d
Gcpk=1 si la clase c pertenece a la subparte p del curso k
M=número muy grande

#Penalizaciones:
PT_d = penalizacion asociada al dia d
PS_s=penalizacion asociada a la sala s
PD_r=penalizacion asociada a la distribucion r
PE=penalizacion asociada a conflictos estudiantiles

#Variables
Xctd=1 si se asigna la clase c en el slot t del dia d
Ycstd=1 si la clase c se asigna a la sala s en el slot t del dia d
Zek=1 si el estudiante e esta asignado al curso k
Wec=1 si el estudiante e esta asignado a la clase c
Auxn=binaria auxiliar para n=0,1,... N

x = {} 
for c in dicc_cluster.keys():
    for t in dicc_dias.keys():
        for b in dicc_bloques.keys():
            x[c,d,b] = timetabling.addVar(vtype=gp.GRB.BINARY,name='x_'+str(c)+','+str(d)+','+str(b))
y = {}
for c in dicc_cluster.keys():
    for z in dicc_salas.keys():
        y[c,z] = timetabling.addVar(vtype=gp.GRB.BINARY,name='y_'+str(c)+','+str(z))
z = {} 
for c in dicc_cluster.keys():
    for t in dicc_dias.keys():
        for b in dicc_bloques.keys():
            x[c,d,b] = timetabling.addVar(vtype=gp.GRB.BINARY,name='x_'+str(c)+','+str(d)+','+str(b))
w = {}
for c in dicc_cluster.keys():
    for z in dicc_salas.keys():
        y[c,z] = timetabling.addVar(vtype=gp.GRB.BINARY,name='y_'+str(c)+','+str(z))


#Función Objetivo
#timetabling.setObjective(gp.quicksum(y[c,z] for c in C for z in Z),GRB.MINIMIZE)

#Restricciones

#1. Fijar la clase c en el slot t del dia d (No hace nada realmente, pero falta ver si puede llegar a hacer algo)
timetabling.addConstrs(((x[c,t,d]<= 1) for c in C for t in T for d in D), name = "R1")
###2. La suma de las clases de un mismo curso debe ser igual al parámetro asociado (sumatoria en t? o para todo t?)
timetabling.addConstrs(((gp.quicksum(G[c,p,k]*x[c,t,d] for c in C)<= 1) for p in P for k in K for t in T for d in D), name = "R2")
#3. Cada clase c que se imparte tiene que tener asignada una sala s
timetabling.addConstrs((x[c,t,d] == y[c,s,t,d]) for c in C for t in T for d in D for S in S, name = "R3")
#4. Respetar capacidad
timetabling.addConstrs(((L[c]*x[c,t,d]<= y[c,s,t,d]*Q[s]) for c in C for t in T for d in D for s in S), name = "R4") 
#5. Respetar la duración de la clase
timetabling.addConstrs(((gp.quicksum(x[c,t,d] for t in T) == DT[c,d]) for c in C for d in D), name = "R5")
#6. A cada alumno se le debe ser asignado los cursos que demandó
timetabling.addConstrs(((gp.quicksum(z[e,k] for k in K) == QK[e]) for e in E), name = "R6")
#7. La suma de los estudiantes asignados a una clase debe ser menor o igual a la capacidad de la clase
timetabling.addConstrs(((gp.quicksum(w[e,c] for e in E) <= L[c]) for c in C), name = "R7")
#8. No sabemos cómo hacer una restricción que tenga que ver con la duración de una clase (pensamos en hacerlo con alguna variable)
timetabling.addConstrs(((gp.quicksum(x[c,t,d] for t in t+DT[c,d]) == DT[c,d]) for c in C for d in D), name = "R8")

# Restricciones (distribuciones)
#Same Start
timetabling.addConstrs(((x[c1,t1,d]<= x[c2,t2,d] for d in D), name = "R9") 
#Same Time
timetabling.addConstrs(((aux_0+aux_1== 1, name = "R10")
timetabling.addConstrs(((x[c1,t,d] + M*aux_0 <= x[c2,t,d] + M for t in T for d in D), name = "R11")
timetabling.addConstrs(((x[c2,t+DT[c1,d],d] + M*aux_0 <= x[c2,t+DT[e,d],d] for t in T for d in D for e in E), name = "R12")
timetabling.addConstrs(((x[c2,t,d] + M*aux_1 <= x[c1,t,d] + M for t in T for d in D), name = "R13")
timetabling.addConstrs(((x[c1,t+DT[e,d],d] + M*aux_1<= x[c2,t+DT[e,d],d] + M for t in T for d in D for e in E), name = "R14")
#Different Time
timetabling.addConstrs(((aux_2+aux_3== 1 , name = "R15")
timetabling.addConstrs(((x[c1,t+DT[c1,d],d] + M*aux_2 <= x[c2,t,d] + M for t in T for d in D, name = "R16")
timetabling.addConstrs(((x[c2,t+DT[c2,d],d] + M*aux_3 <= x[c1,t,d] + M for t in T for d in D, name = "R17")   
                                                                                             

timetabling.optimize()
for v in timetabling.getVars():
    print('%s %g' % (v.varName, v.x))


SyntaxError: invalid syntax (<ipython-input-18-f41159961e84>, line 24)